<h2>Setting Stuff Up</h2>

Here we import some packages that we'll need in various places. We'll also load all the variables we set in config.

In [ ]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
import json
from agavepy.agave import Agave
from setvar import *
from time import sleep
import json

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
setvar("""
AGAVE_APP_NAME=training-${AGAVE_USERNAME}
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-${AGAVE_USERNAME}
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-${AGAVE_USERNAME}
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
AGAVE_STORAGE_HOME_DIR=/home/${MACHINE_USERNAME}
SCRATCH_DIR=/home/${MACHINE_USERNAME}
AGAVE_STORAGE_WORK_DIR=/home/${MACHINE_USERNAME}
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
""")
loadvar()

## Initializing the CLI

If you are running against a privately hosted Agave tenant, you need to configure the tutorial to run against your tenant. You can do that by setting the `AGAVE_TENANTS_API_BASEURL` and `AGAVE_TENANT_ID` environment variables. 

If you specified the correct address to your Tenants API, you should be able to discover the tenants available for you through the CLI.

In [ ]:
!tenants-list

Select the tenant you would like to use by setting the `AGAVE_TENANT` environment variable. You may select either the name of the tenant, or leave it blank to select the default tenant.

In [ ]:
!tenants-init -t sandbox

### Creating Client API Keys

In this next step we delete the client if it exists. Chances are, yours doesn't yet. We put this command here in case, for some reason, you want to re-create your client later on. If you delete the client you intend to create before you create it, no harm is done.

In [ ]:
!clients-delete -u "$AGAVE_USERNAME" -p "$AGAVE_PASSWORD" $AGAVE_APP_NAME

In this step we create the client. Clients provide a way of encapsulating resources connected to a single project. Through the client, you will receive a token which you can use to run most of the Agave commands.

In [ ]:
!clients-create -u "$AGAVE_USERNAME" -p "$AGAVE_PASSWORD" -N "$AGAVE_APP_NAME" -S

Create the token for your client. You will, from this point on, use this token to run the remainder of the Agave commands in this tutorial.

In [ ]:
!auth-tokens-create -u $AGAVE_USERNAME -p "$AGAVE_PASSWORD" 

## FOLLOWING ALONG AT HOME  

If you are following along at home using the docker-compose stack, you will need to run the following cell to get the hostname and port of your tcp tunnel so Agave can contact your system without a public IP address.

In [ ]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||'
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['VM_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['VM_MACHINE'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_HOST'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_PORT'] = readfile('ngrok_port.txt').strip()
    !echo "VM_PORT=$VM_PORT"
    !echo "VM_MACHINE=$VM_MACHINE"
    setvar("VM_IPADDRESS=$(getent hosts ${VM_MACHINE}|cut -d' ' -f1)")

## Creating a Storage System   

Agave wants to know which place (or places) you want to store the data associated with your jobs. Here, we're going to set that up. Authentication to the storage machine will be through SSH keys. The key and public key files, however, contain newlines. To encode them in Json (the data format used by Agave), we will run the jsonpki command on each file. Next, we will store its contents in the environment for use by setvar.

In [ ]:
!mkdir -p ~/key
!chmod 700 ~/key
!jsonpki --public ~/.ssh/id_rsa.pub > ~/key/id_rsa.pub.txt
!jsonpki --private ~/.ssh/id_rsa > ~/key/id_rsa.txt

In [ ]:
os.environ["PUB_KEY"]=readfile("${HOME}/key/id_rsa.pub.txt").strip()
os.environ["PRIV_KEY"]=readfile("${HOME}/key/id_rsa.txt").strip()

In this next cell, we create the json file used to describe the storage machine.

In [ ]:
writefile("${AGAVE_STORAGE_SYSTEM_ID}.txt","""{
    "id": "${AGAVE_STORAGE_SYSTEM_ID}",
    "name": "${MACHINE_NAME} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "type": "STORAGE",
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    }
}
""")

Here, we tell Agave about the machine. You can re-run the previous cell and the next one if you want to change the definition of your storage machine.

In [ ]:
!systems-addupdate -F ${AGAVE_STORAGE_SYSTEM_ID}.txt

Next we run the Agave command `files-list`. This provides a check that we've set up the storage machine correctly.

In [ ]:
!files-list -l 5 -S ${AGAVE_STORAGE_SYSTEM_ID} 

## Creating an Execution System

You may not always wish to store your data on the same machine you run your jobs on. However, in this tutorial, we will assume that you do. The description for the execution machine is much like the storage machine. However, there are a few more pieces of information you'll need to provide. In this example, we are going to call commands directly on the host as opposed to using a batch queue scheduler. It is slightly simpler.

In [ ]:
# Edit any parts of this file that you know need to be changed for your machine.
writefile("${AGAVE_EXECUTION_SYSTEM_ID}.txt","""
{
    "id": "${AGAVE_EXECUTION_SYSTEM_ID}",
    "name": "${MACHINE_NAME} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "CLI",
    "scheduler" : "FORK",
    "environment": null,
    "scratchDir" : "${SCRATCH_DIR}",
    "queues": [
        {
            "name": "none",
            "default": true,
            "maxJobs": 10,
            "maxUserJobs": 10,
            "maxNodes": 6,
            "maxProcessorsPerNode": 6,
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "00:30:00"
        }
    ],
    "login": {
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        },
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    },
    "workDir": "${AGAVE_STORAGE_WORK_DIR}"
}""")

In [ ]:
!systems-addupdate -F ${AGAVE_EXECUTION_SYSTEM_ID}.txt

In [ ]:
# Test to see if this worked...
!files-list -l 5 -S ${AGAVE_EXECUTION_SYSTEM_ID} .

## Create your First App

Agave allows us to describe custom allocations, limiting users to run a specific job. In this case, we're going to create a simple "fork" scheduler that just takes the command we want to run as a job parameter. The wrapper file is a shell script we will run on the execution machine. If we were using a scheduler, this would be our batch file.

In [ ]:
writefile("fork-wrapper.txt","""
#!/bin/bash
\${command}
""")

Using Agave commands, we make a directory on the storage server an deploy our wrapper file there.

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-wrapper.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}/

All agave applications require a test file. The test file is a free form text file which allows you to specify what resources you might need to test your application.

In [ ]:
writefile("fork-test.txt","""
command=date
fork-wrapper.txt
""")

In [ ]:
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-test.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}/

Like everything else in Agave, we describe our application with Json. We specifiy which machines the application will use, what method it will use for submitting jobs, job parameters and files, etc.

In [ ]:
writefile("fork-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-fork",
   "version":"1.0",
   "label":"Runs a command",
   "shortDescription":"Runs a command",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"${AGAVE_APP_DEPLOYMENT_PATH}",
   "templatePath":"fork-wrapper.txt",
   "testPath":"fork-test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[
         {   
         "id":"datafile",
         "details":{  
            "label":"Data file",
            "description":"",
            "argument":null,
            "showArgument":false
         },
         "value":{  
            "default":"/dev/null",
            "order":0,
            "required":false,
            "validator":"",
            "visible":true
         }
      }   
   ],
   "parameters":[{
     "id" : "command",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"/bin/date",
       "validator":null
     },
     "details":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")

In [ ]:
!apps-addupdate -F fork-app.txt

## Running  Jobs

Now that we have specified our application using Agave, it is time to try running jobs. To start a job we, once again, create a Json file. The Json file describes the app, what resource to run on, as well as how and when to send notifications. Notifications are delivered by callback url. EMAIL is the easiest type to configure, but we show here how to send webhook notifications to the popular [RequestBin](https://requestb.in/). 

Before we configure our notification, we need to create a requestbin to use. There are convenience commands to interact with requestbin built into the Agave CLI. We will use those to get our URL.

In [ ]:
rburl = !requestbin-create 
os.environ['REQUESTBIN_URL'] = rburl[0].strip()

Now that we have a URL to recieve webhooks from our job, Let's look at our job request. The way this job is configured, it will send the requestbin notifications for every job event until the job reaches a terminal state. For a full list of job events, please see http://docs.agaveplatform.org/#job-monitoring

In [ ]:
writefile("job.txt","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "executionSystem": "${AGAVE_EXECUTION_SYSTEM_ID}",
   "archive": false,
   "notifications": [
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&jobid=\${JOB_ID}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "command":"echo hello"
   }
 }
""")

Because the setvar() command can evalute `$()` style bash shell substitutions, we will use it to submit our job. This will capture the output of the submit command, and allow us to parse it for the JOB_ID. We'll use the JOB_ID in several subsequent steps.

In [ ]:
setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F job.txt)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")

### Job Monitoring and Output

While the job is running, the requestbin you registered will receive webhooks from Agave every time a job event occurs. To monitor this in real time, evaluate the next cell an visit the printed url in your browser:

In [ ]:
print ('%s?inspect'%os.environ['REQUESTBIN_URL'])

Of course, you can also monitor the job status by polling. Note that the notifications you receive via email and webhook are less wasteful of resources. However, we show you this for completeness.

In [ ]:
for iter in range(20):
    setvar("STAT=$(jobs-status $JOB_ID)")
    stat = os.environ["STAT"]
    sleep(5.0)
    if stat == "FINISHED" or stat == "FAILED":
        break

The jobs-history command provides you a record of the steps of what your job did. If your job fails for some reason, this is your best diagnostic.

In [ ]:
!jobs-history ${JOB_ID}

This command shows you the job id's and status of the last 5 jobs you ran.

In [ ]:
!jobs-list -l 5

This next command provides you with a list of all the files generated by your job. You can use it to figure out which files you want to retrieve with jobs-output-get.

In [ ]:
!jobs-output-list --rich --filter=type,length,name ${JOB_ID}

Retrieve the standard output.

In [ ]:
!jobs-output-get ${JOB_ID} fork-command-1.out
!cat fork-command-1.out

Retrieve the standard error output.

In [ ]:
!jobs-output-get ${JOB_ID} fork-command-1.err
!cat fork-command-1.err

<h3>Automating</h3>
Because we're working in Python, we can simply glue the above steps together and create a script to run jobs for us and fetch the standard output. Let's do that next.

In [ ]:
%%writefile runagavecmd.py
from setvar import *

from time import sleep

def runagavecmd(cmd,infile=None):
    setvar("REMOTE_COMMAND="+cmd)
    setvar("REQUESTBIN_URL=$(requestbin-create)")
    print("")
    print(" ** QUERY STRING FOR REQUESTBIN **")
    print('%s?inspect'%os.environ['REQUESTBIN_URL'])
    print("")
    # The input file is an optional parameter, both
    # to our function and to the Agave application.
    if infile == None:
        setvar("INPUTS={}")
    else:
        setvar('INPUTS={"datafile":"'+infile+'"}')
    setvar("JOB_FILE=job-remote-$PID.txt")
    # Create the Json for the job file.
    writefile("$JOB_FILE","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "executionSystem": "${AGAVE_EXECUTION_SYSTEM_ID}",
   "archive": false,
   "notifications": [
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&jobid=\${JOB_ID}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "command":"${REMOTE_COMMAND}"
   },
   "inputs":${INPUTS}
 }""")
    # Run the job and capture the output.
    setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F $JOB_FILE)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")
    if not re.match(r'^[\w-]+$',os.environ['JOB_ID']):
        print("Bad JOB_ID:",os.environ['JOB_ID'])
        return
    # Poll and wait for the job to finish.
    for iter in range(80): # Excessively generous
        setvar("STAT=$(jobs-status $JOB_ID)")
        stat = os.environ["STAT"]
        sleep(5.0)
        if stat == "FINISHED" or stat == "FAILED":
            break
    # Fetch the job output from the remote machine
    setvar("CMD=jobs-output-get ${JOB_ID} fork-command-1.out")
    os.system(os.environ["CMD"])
    print("All done! Output follows.")
    # Load the output into memory
    output=readfile("fork-command-1.out")
    print("=" * 70)
    print(output)

In [ ]:
import runagavecmd as r
import imp
imp.reload(r)

In [ ]:
r.runagavecmd("lscpu")

<h2>Permissions and Sharing</h3>

List the users and the permssions they have to look at the given job.

In [ ]:
!jobs-pems-list ${JOB_ID}

In [ ]:
# now pair off with your neighbor and both of you share your job with them.
# For now, just give read access
!jobs-pems-update -u training002 -p READ ${JOB_ID}

In [ ]:
# Now let's see if we can see our neighbor's job
# Now let's see if we can see our neighbor's job
shared_job = !jobs-search --filter=id -l 1 owner.neq=${AGAVE_USERNAME} 
os.environ['SHARED_JOB_ID'] = shared_job[0]
print(os.environ['SHARED_JOB_ID'])

Permissions are just that, permitting someone to do something. You said your neighbor could view your job. Let's see what that means.

In [ ]:
# You already searched for the job and found it, so you should be able to lis
# an view the details
!jobs-list $SHARED_JOB_ID

In [ ]:
# You should also be able to view the history. Here we'll just return the last few 
# events. Notice the history event showed up history event
!jobs-history --limit 3 --order desc $SHARED_JOB_ID

In [ ]:
# You can also view their job output
!jobs-output-list -L $SHARED_JOB_ID

In [ ]:
# What if we no longer want to see the job. Let's delete it.
!jobs-delete $SHARED_JOB_ID

Doah! We can't delete the shared job because we weren't granted write permission.

In [ ]:
# Let's grant write access and see what we can do
!jobs-pems-update -u training002 -p READ_WRITE ${JOB_ID}

In [ ]:
# Now let's see if we can delete the shared job
!jobs-delete $SHARED_JOB_ID

In [ ]:
# Wait, now we don't have anything to work with. 
# No worries. Agave doens't really delete anything. Your job is still there
# We just need to restore it.
!jobs-restore $SHARED_JOB_ID

In [ ]:
# Now let's try to rerun the job
!jobs-resubmit $SHARED_JOB_ID

In [ ]:
# Well, what app did they use in the job?

shared_job = !jobs-list -v --filter=executionSystem,appId $SHARED_JOB_ID | jq -r '. | [ .executionSystem , .appId] | .[]'
print(shared_job)
os.environ['SHARED_JOB_APP'] = shared_job[1]
os.environ['SHARED_JOB_SYSTEM'] = shared_job[0]

In [ ]:
# Hmm, do we have access to the app?
! apps-pems-list $SHARED_JOB_APP

In [ ]:
# Oh, we don't have permission to even view the app. Guess our job permissions
# don't extend to the application. Let's be a good neighbor and share our apps
# with each other
! apps-pems-update -u training002 -p READ "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0"

In [ ]:
# Now do we have access to the app?
! apps-pems-list $SHARED_JOB_APP

In [ ]:
# Score, but wait, do I need execute to run? We should granb that too.
# Hmm, do we have access to the app?
! apps-pems-update -u training002 -p EXECUTE "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0"

In [ ]:
# Now do we have access to the app?
! apps-pems-list $SHARED_JOB_APP

In [ ]:
# I guess permissions aren't hierachical. Now i can execute it (I think), but I can't
# read it. How aabout we grant read_execute instead
! apps-pems-update -u training002 -p READ_EXECUTE "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0"

In [ ]:
# Now do we have access to the app?
! apps-pems-list $SHARED_JOB_APP

In [ ]:
# So now we can rerun our neighbor's job, right
!jobs-resubmit -v $SHARED_JOB_ID

In [ ]:
# drat. why can't we run now? Do we have system access?
!systems-roles-list $SHARED_JOB_SYSTEM

In [ ]:
# ok, let's skip to the end. we'll just realize we should grant a user rather than guest role
# to the system
!systems-roles-addupdate -u training002 -r USER $AGAVE_EXECUTION_SYSTEM_ID

In [ ]:
# that should work, right?
!systems-roles-list $SHARED_JOB_SYSTEM

In [ ]:
# So can we run the job now?
resubmitted_job_id = ! jobs-resubmit -v --filter=id $SHARED_JOB_ID | jq -r '.id'
os.environ['RESUBMITTED_JOB_ID'] = resubmitted_job_id[0]

In [ ]:
# yay. wait, who owns the data?
print (resubmitted_job_id[0])
! jobs-pems-list $RESUBMITTED_JOB_ID

In [ ]:
# mine, mine, mine, mine, mine, mine, mine, mine, mine, mine, mine, mine, mine
# kill it, we're moving on.
! jobs-stop $RESUBMITTED_JOB_ID

In [ ]:
# we can also share data a few ways
job_output_url = ! jobs-output-list -v --filter=_links $JOB_ID fork-command-1.out | jq -r '.[0]._links.self.href'
os.environ['JOB_OUTPUT_URL'] = job_output_url[0]

In [ ]:
postit_url = ! postits-create -m 3 -l 86400 -V $JOB_OUTPUT_URL | jq -r '.result._links.self.href' 

In [ ]:
# click on the link a few times to see it working.
print (postit_url[0])

In [ ]:
# you can also share your data via the files api
# let's share the job directory with each other
job_path = ! jobs-list -v $JOB_ID | jq -r '.outputPath'
os.environ['JOB_OUTPUT_FOLDER_PATH'] = job_path[0]
! files-pems-update -u training002 -p read -S $AGAVE_EXECUTION_SYSTEM_ID $JOB_OUTPUT_FOLDER_PATH/fork-command-1.out

In [ ]:
!jobs-delete $JOB_ID

## Managing Data

You can also use Agave to manage your data


## Metadata & Tagging

Agave provides two different ways to create and manage custom metadata and realationships between your resources. The Metadata API provides a searchable key-value store that supports structured as well as binary data. Let's see how this works by adding some metadata about our app.

In [67]:
writefile("fork-app-metadata.json", """
{
  "name": "notes",
  "value": [
      "Taking first steps with the Agave Platform CLI. This is pretty cool."
  ]
}
""")
setvar("""
APP_METADATA_ID=$(metadata-addupdate -v -F fork-app-metadata.json | jq -r '.uuid')
""")

Writing file `fork-app-metadata.json'
APP_METADATA_ID=4199900087058895336-242ac11d-0001-012


The Tags API works similar to the metadata, but enforces a unique per-user namespace on the tag names. Tagging is an option API, and as such, is not enabled for new API clients by default in every tenant. 

> You can subscribe to this and other boutique APIs by running the folling command.

In [22]:
%%bash
# echo $AGAVE_APP_NAME
# clients-list 

# for i in tags uuids tenants; 
# do 
#     echo "clients-subscriptions-update -V -N $i $AGAVE_APP_NAME"
#     clients-subscriptions-update -V -N $i "$AGAVE_APP_NAME"
# done

clients-subscriptions-list --limit=50 "$AGAVE_APP_NAME"
auth-check

Apps
Files
Jobs
Meta
Monitors
Notifications
Postits
Profiles
Systems
Transforms
tenant: sandbox
username: dooley
time left: 14377 seconds
expires at: Sat Jul 21 22:26:25 UTC 2018


Your current token will not have access to these newly subscribed APIs, so we need to 
revoke our current token and get a fresh one.

In [20]:
!auth-tokens-revoke
!auth-tokens-create

Token 4fb3e5ad3c469263039c4d78af7cca7 successfully revoked
Token for sandbox:dooley successfully refreshed and cached for 14400 seconds
9632189215d348e4ba1cb48c9785f6ab


In [68]:
!tags-list

## Events and Notificaitons 

Earlier we saw how we could subscribe for webhook job status updates to be sent to a RequestBin. Job status updates are one of the many events Agave throws. Other examples are file updates, metadata creation, user creation, system outages. 

> You can see the current list of subscribable events broken down by resource in the [Agave Platform Event Reference](https://docs.agaveplatform.org/#event-reference) section of the [Agave Developer Docs](https://docs.agaveplatform.org/).

That is one of several delivery methods supported by Agave. Agave also supports:  

* Slack 
* Email 
* Websocket
* HTTP GET/POST
* Event triggers
* Internal URL
* Persistent queues

We will look at some of these later on when we talk about automation and publishing. Right now, let's look at some examples with our current examples.

### Persistent vs transient notifications

When you create a notification, you can subscribe to the first occurrence of an event or to every even on a resources. Our previous jobs used persistent subscriptions. One notification subscription was created with the job, a wildcard, `*` was used as the named event, meaning "subscribe this requestbin to notifications for any event thrown by this job," and `persistent` was set to true, meaning that the subscription will stay active regardless of how often it fires. 

If we did not want to get quite so many notifications sent to us, we could create one notifications for each individual event that we care about. The following is a job that illustrates several different notification types. 

In [95]:
# !echo $AGAVE_EXECUTION_SYSTEM_ID
# writefile("$HOME/work/job-with-many-notifications-exe.txt","""
# #!/usr/bin/env bash
# ]echo "Inside job \${AGAVE_JOB_ID}

# # Make runtime callback event to write a metadata item 
# # from inside the job.
# \${AGAVE_JOB_CALLBACK_NOTIFICATION|JOB_METADATA_EVENT|name:JOB_NAME,value:JOB_APP_ID,associatedUuid:JOB_ID}
# "inputs": {
#        "datafile": "agave://$AGAVE_EXECUTION_SYSTEM_ID/work/job-with-many-notifications-exe.txt"
#    },
# ,
#     {
#       "url": "$(auth-check -v | jq -r '.baseurl')/meta/v2/data",
#       "event": "JOB_METADATA_EVENT",
#       "persistent":"false",
#       "policy": {
#          "retryStrategy": "DELAYED",
#          "retryLimit": 2,
#          "retryRate": 5,
#          "retryDelay": 10,
#          "saveOnFailure": true
#        }
#     }
# """)

writefile("job-with-many-notifications.txt","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "archive": false,
   "notifications": [
    {
      "url":"${SLACK_WEBHOOK_URL}",
      "event":"RUNNING",
      "persistent": false
    },
    {
      "url":"${SLACK_WEBHOOK_URL}",
      "event":"FINISHED",
      "persistent": false
    },
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&owner=\${OWNER}&id=\${JOB_ID}",
      "event":"CLEANING_UP",
      "persistent": false
    },
    {
      "url":"demo+spam@agaveplatform.org",
      "event":"CLEANING_UP",
      "persistent": false
    },
    {
      "url":"https://httpbin.org/500",
      "event":"*",
      "persistent":"true",
      "policy": {
         "retryStrategy": "NONE",
         "saveOnFailure": true
       }
    }
   ],
   "parameters": {
     "command":"date && echo hello"
   }
 }
""")

# setvar("""
# # Capture the output of the job submit command
# OUTPUT=$(jobs-submit -W -F job-with-many-notifications.json)
# # Parse out the job id from the output
# JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
# """)

Writing file `job-with-many-notifications.txt'


In [96]:
setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F job-with-many-notifications.txt)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")
for iter in range(20):
    setvar("STAT=$(jobs-status $JOB_ID)")
    stat = os.environ["STAT"]
    sleep(5.0)
    if stat == "FINISHED" or stat == "FAILED":
        break
!jobs-history $JOB_ID

OUTPUT=Successfully submitted job 803280584174726680-242ac114-0001-007
JOB_ID=803280584174726680-242ac114-0001-007
STAT=STAGED
STAT=SUBMITTING
STAT=SUBMITTING
STAT=SUBMITTING
STAT=SUBMITTING
STAT=SUBMITTING
STAT=FINISHED
Job accepted and queued for submission.
Skipping staging. No input data associated with this job.
Preparing job for submission.
Attempt 1 to submit job
Fetching app assets from agave://sandbox-storage-dooley/agave-deploy
Staging runtime assets to agave://sandbox-exec-dooley//home/jovyan/dooley/job-803280584174726680-242ac114-0001-007-fork-command-1
CLI job successfully forked as process id 6535
CLI job successfully forked as process id 6535
Job receieved duplicate RUNNING notification
Job completed execution
Job completed. Skipping archiving at user request.


In [97]:
!jobs-output-list --rich $JOB_ID

| name                  | length | permission | type | lastModified           |
| ----                  | ------ | ---------- | ---- | ------------           |
| .agave.archive        | 78     | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| .agave.log            | 395    | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| fork-command-1.err    | 0      | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| fork-command-1.ipcexe | 2499   | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| fork-command-1.out    | 35     | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| fork-command-1.pid    | 5      | READ_WRITE | file | Jul 21, 2018   8:17 pm |
| fork-test.txt         | 29     | READ_WRITE | file | Jul 21, 2018   7:33 pm |
| fork-wrapper.txt      | 22     | READ_WRITE | file | Jul 21, 2018   7:33 pm |
| input.txt             | 1753   | READ_WRITE | file | Jul 21, 2018  12:41 am |


In [98]:
!jobs-output-get -P $JOB_ID fork-command-1.out

Sun Jul 22 01:17:47 UTC 2018
hello


When this job runs, you will see notifications sent to:

* The `#training-webhooks` channel on the [Agave Platform Slack Team](https://slackin.agaveplatform.org) when the job starts running and enters a finished state.  
* Your RequestBin when the job's compute process exits and Agave starts the cleanup phase of the job lifecycle.  
* An email to a autoresponder email account.  

The last two notifications aren't quite as straight forward. The first one will send a message to `httpbin`, a http mirror service that allows you to craft URL that return predetermined content. The `https://httpbin.org/500` we subscribe to the wildcard job event will always return a [500 error code], signaling a failed delivery. We use this example to show a retry attempt policy. The `policy` object of this notifiation says to quit after the first attempt without any subsequent retires and immediately store the failed notification attempt information in the dead letter queue for the notification where it can be queried later on.


In [102]:
!echo "${REQUESTBIN_URL}?inspect"

https://requestbin.agaveapi.co/1880ovh1?inspect


In [103]:
%%bash

HTTPBIN_JOB_NOTIFICATION_ID=$(notifications-search -l 1 -v associatedUuid=$JOB_ID url=https://httpbin.org/500 | jq -r '.id')
notifications-list-failures $HTTPBIN_JOB_NOTIFICATION_ID

Invalid value for event.
stty: 'standard input': Inappropriate ioctl for device

Please specify a valid notification object id to query for failures
stty: 'standard input': Inappropriate ioctl for device



Jobs are not the only things that throw or receive job events. Perhaps we want to track the number of times our app, or any app is run on a system. We can subscribe for an event every time a job is run on the system.

In [ ]:
setvar("""
AGAVE_EXECUTION_SYSTEM_UUID=$(systems-list --filter=uuid -v $AGAVE_EXECUTION_SYSTEM_ID | jq '.uuid')
""")

writefile("system-job-notification.json", """
{
  "url":"https://httpbin.org/500",
  "event":"JOB_SUBMIT",
  "persistent":"true",
  "associatedUuid": "$AGAVE_EXECUTION_SYSTEM_UUID",   
  "policy": {
     "retryStrategy": "NONE",
     "retryLimit": 2,
     "retryRate": 1,
     "retryDelay": 0,
     "saveOnFailure": true
   }
}
""")


Agave can watch for events on data it touches directly and indirectly. Here we subscribe to notifications on changes to a specific file, our app wrapper template. 

In [ ]:
setvar("""
FILE_ITEM_UUID=$(files-list --filter=* -v -S AGAVE_STORAGE_SYSTEM_ID $AGAVE_APP_DEPLOYMENT_PATH/fork-wrapper.txt | jq '.uuid')
""")

writefile("file-overwritten-notification.json", """
{
  "url":"${REQUESTBIN_URL}?event=\${EVENT}&path=\${PATH}",
  "event":"OVERWRITTEN",
  "persistent":"false",
  "associatedUuid": "${FILE_ITEM_UUID}"
}
""")
fileOverwriteNotification = !notifications-addupdate -v -F file-overwritten-notification.json

We can also subscribe to changes to our app deployment directory. 

In [ ]:
setvar("""
DEPLOYMENT_DIRECTORY_UUID=$(files-list -l 1 --filter=* -v -S AGAVE_STORAGE_SYSTEM_ID $AGAVE_APP_DEPLOYMENT_PATH | jq '.[0].uuid')
""")

writefile("directory-content-changed-notification.json", """
{
  "url":"${REQUESTBIN_URL}?event=\${EVENT}&path=\${PATH}",
  "event":"CONTENT_CHANGED",
  "persistent":"false",
  "associatedUuid": "${DEPLOYMENT_DIRECTORY_UUID}"
}
""")

directoryChangeNotification = !notifications-addupdate -v -F directory-content-changed-notification.json

Let's upload our job wrapper template to our app's deployment directory again and see what happens

In [ ]:
print ${REQUESTBIN_URL}

Go ahead and create these notification subscriptions by running the following cell.

Now let's upload a file and see what happens.

In [ ]:
print ('%s?inspect'%os.environ['REQUESTBIN_URL'])

a

## Using the Agave ToGo web portal  

Follow the link below to run your job from a web portal.

In [ ]:
!echo http://togo.agaveplatform.org/app/#/apps/${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0/run